In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
TS = pd.read_csv('TS.csv', sep = ';')
sign = 0.05
plt.rcParams['font.family']='serif'
plt.rcParams['font.weight']='light'

In [ ]:
TS.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
var_num = TS.drop(['Campione','Classe'], axis=1)
X_as = StandardScaler().fit_transform(var_num)
df_X_as = pd.DataFrame(X_as, columns = var_num.columns)
#df_X_as

In [ ]:
#Correlation matrix
import seaborn as sns
corrmat = var_num.corr()
hm = sns.heatmap(corrmat,
                 vmin=-1,
                 vmax=1,               
                 cbar=True, 
                 annot=True, 
                 square=True, 
                 fmt='.2f',
                 linewidth=.5,
                 annot_kws={'size': 10}, 
                 yticklabels=var_num.columns, 
                 xticklabels=var_num.columns, 
                 cmap='RdBu') #Spectral_r for more colours
plt.show()
#corrmat.to_csv('corrmat.csv',index=True,sep=';')

In [ ]:
#PCA with sklearn
from sklearn.decomposition import PCA
pca = PCA()
scores = pca.fit_transform(X_as)
PC = pca.components_.T #eigenvectors
exp_var = pca.explained_variance_
exp_var_p= pca.explained_variance_ratio_
cu_var = []
cuvar_ist=0
for i in range(len(exp_var_p)):
  cuvar_ist = cuvar_ist + exp_var_p[i]
  cu_var.append(cuvar_ist) #cumulative explained variance

In [ ]:
#Scree plot
x = list(range(1,len(exp_var_p)+1))
y = cu_var
plt.figure(figsize=(9,5))
plt.plot(x,y,'b',linewidth=1,marker='o')
plt.title('Scree plot')
plt.xlabel('numero di PC')
plt.ylabel('Varianza cumulativa')
plt.grid(True)
plt.show

In [ ]:
#PC filter
cuvar_min = 0.8
nPC = 1
while cu_var[nPC-1] < cuvar_min:
  nPC += 1
L = PC[:,0:(nPC)]
M = PC[:,nPC:] #not used PCs, for Q_critical diagnostic 
T = X_as @ L
print('numero di PC sufficienti =',nPC)
print('varianza cumulativa con %.f PC = %.3f' % (nPC,1*(cu_var[nPC-1])))

In [ ]:
#Scores plot
#aggiungere ellisse di confidenza
PCx = 1
PCy = 2
df_scores = pd.DataFrame(scores, columns = [f'PC{i+1}' for i in range(scores.shape[1])])
df_scores['Classe'] = TS['Classe'].values
plt.figure(figsize=(9,6))
for cat, gruppo in df_scores.groupby('Classe'):
    plt.scatter(gruppo[f'PC{PCx}'], gruppo[f'PC{PCy}'], label=cat)
plt.title('Scores plot')
plt.xlabel(df_scores.columns[PCx - 1])
plt.ylabel(df_scores.columns[PCy - 1])
plt.legend(title='Classe')
plt.grid(True)
plt.show()

In [ ]:
#Loadings plot
#aggiungere etichette PCn
PCx = 1
PCy = 2
df_PC = pd.DataFrame(PC, index = df_X_as.columns, columns = [f'PC{i+1}' for i in range(PC.shape[1])])
x = df_PC.iloc[:,(PCx-1)]
y = df_PC.iloc[:,(PCy-1)]
plt.figure(figsize=(9,6))
for dx, dy in zip(x, y):
    plt.arrow(0, 0, dx, dy,head_length=0.02 , head_width=0.02, overhang=1)
plt.title('Loadings plot')
plt.xlabel(df_PC.columns[PCx - 1])
plt.ylabel(df_PC.columns[PCy - 1])
plt.grid(True)
for i in range(PC.shape[0]):
    plt.text(x.iloc[i], y.iloc[i], x.index[i], fontsize=8, color='black')
plt.show()

In [ ]:
#Biplot
PCx = 1
PCy = 2
df_scores = pd.DataFrame(scores, columns = [f'PC{i+1}' for i in range(scores.shape[1])])
df_PC = pd.DataFrame(PC, index = df_X_as.columns, columns = [f'PC{i+1}' for i in range(PC.shape[1])])
df_scores['Classe'] = TS['Classe'].values
plt.figure(figsize=(9,6))
for cat, gruppo in df_scores.groupby('Classe'):
    plt.scatter(gruppo[f'PC{PCx}'], gruppo[f'PC{PCy}'], label=cat)
x = 6*df_PC.iloc[:,(PCx-1)]
y = 6*df_PC.iloc[:,(PCy-1)]
for dx, dy in zip(x, y):
    plt.arrow(0, 0, dx, dy,head_length=0.2 , head_width=0.2, overhang=1)
plt.title('Biplot Scores e Loadings')
plt.xlabel(df_scores.columns[PCx - 1])
plt.ylabel(df_scores.columns[PCy - 1])
plt.legend(title='Classe')
plt.grid(True)
for i in range(PC.shape[0]):
    plt.annotate(x.index[i], xy=(x.iloc[i], y.iloc[i]), xytext=((x.iloc[i]+0.2),(y.iloc[i]+0.2)))
plt.show()

In [ ]:
#Diagnostic and influence plot
from scipy.stats import f
from scipy.stats import norm

n = T.shape[0] 
T_cov = np.diag(exp_var[:(nPC)])

T2 = np.einsum('ij,jk,ik->i', T, np.linalg.inv(T_cov), T)
T2_F = f.ppf((1 - sign/2), nPC, (n - nPC))
T2_crit = (nPC * (n - 1)/(n - nPC)) * T2_F
df_T2 = pd.DataFrame(T2, index=TS['Campione'], columns = ['T^2'])
T2_out = []
T2_out_str = []
for i in range(T2.shape[0]):
    if T2[i] > T2_crit:
        T2_out.append(df_X_as.index[i])
        T2_out_str.append(df_T2.index[i])

E = X_as - (T@(L.T)) #residuals created by the application of the PC filter
#Q = np.diag(E@(E.T))
Q = np.sum(E**2, axis=1) #more efficient
df_Q = pd.DataFrame(Q, index=TS['Campione'], columns = ['Q'])
#Q_crit diagnostic: approx. with Jackson-Mudholkar formula.
Lambda_M = exp_var[(nPC):]
Theta1 = np.sum(Lambda_M)
Theta2 = np.sum(Lambda_M**2)
Theta3 = np.sum(Lambda_M**3)
h0 = 1 - ((2 * Theta1 * Theta3)/(3 * (Theta2**2)))
c_alpha = norm.ppf(1 - sign/2)
Q_crit = Theta1 * ((((c_alpha*np.sqrt(2*Theta2*(h0**2)))/Theta1)+1+((Theta2*h0*(h0-1))/(Theta1**2)))**(1/h0))
Q_out = []
Q_out_str = []
for i in range(Q.shape[0]):
    if Q[i] > Q_crit:
        Q_out.append(df_X_as.index[i])
        Q_out_str.append(df_Q.index[i])

print(len(T2_out) ,'outliers per T^2 fuori soglia %.2f:' % (sign), T2_out_str)
print(len(Q_out) ,'outliers per Q fuori soglia %.2f:' % (sign), Q_out_str)

plt.figure(figsize=(10,7))
plt.plot(df_T2,df_Q,'b',linewidth=0,marker='o')
plt.axvline(T2_crit, linestyle = '--', c = 'red', label = '0.05')
plt.axhline(Q_crit, linestyle = '--', c = 'red', label = '0.05')
plt.title('Influence plot')
plt.xlabel('T^2')
plt.ylabel('Q-residuals')
plt.grid(True)
for i in range(n):
    if T2[i]>T2_crit or Q[i]>Q_crit:
        #plt.annotate(df_T2.index[i], xy=(T2[i],Q[i]), xytext=((T2[i]+0.4),(Q[i]+0.3)), arrowprops=dict(arrowstyle='-', color='black', lw=2))
        plt.annotate(df_T2.index[i], xy=(T2[i],Q[i]), xytext=((T2[i]*1.05),(Q[i]*1.05)), arrowprops=dict(arrowstyle='-', color='black', lw=2))
plt.show

In [ ]:
#Contribution plot for T^2
n_outlier = 1
print (T2_out_str[n_outlier-1])
outlier = T2_out[n_outlier-1]
t = T@np.linalg.inv(np.sqrt(T_cov))@L.T
df_t = pd.DataFrame(t, index = df_X_as.index, columns = var_num.columns)
df_t_i = df_t.iloc[outlier,]
plt.figure(figsize=(9,5))
plt.title('Contribution plot per T^2, %.20s' % (T2_out_str[n_outlier-1]))
plt.barh(df_t.columns, df_t_i)
plt.grid(True)
plt.show

In [ ]:
#Contribution plot for Q
n_outlier = 1
print (Q_out_str[n_outlier-1])
outlier = Q_out[n_outlier-1]
df_E = pd.DataFrame(E, index = df_X_as.index, columns = var_num.columns)
df_E_i = df_E.iloc[outlier,]
plt.figure(figsize=(9,5))
plt.title('Contribution plot per Q index, %.20s' %(Q_out_str[n_outlier-1]))
plt.barh(df_E.columns, df_E_i)
plt.grid(True)
plt.show

In [ ]:
#Projection of QC samples on the scores plot
#aggiungere ellisse di confidenza
QC = pd.read_csv('QC.csv', sep = ';')
var_num_QC = QC.drop(['Campione','Classe'], axis=1)
Mean_TS = np.mean(var_num,axis=0)
STDV_TS = np.std(var_num,axis=0)
QC_scal = (var_num_QC-Mean_TS)/STDV_TS #QC scaling with TS autoscaling values
A_QC = QC_scal @ PC
A_QC['Classe'] = QC['Classe'].values
PCx = 1
PCy = 2
df_scores = pd.DataFrame(scores, columns = [f'PC{i+1}' for i in range(scores.shape[1])])
df_scores['Classe'] = TS['Classe'].values
plt.figure(figsize=(9,6))
for cat, gruppo in df_scores.groupby('Classe'):
    plt.scatter(gruppo[f'PC{PCx}'], gruppo[f'PC{PCy}'], label=cat)
#plt.scatter(df_scores[f'PC{PCx}'], df_scores[f'PC{PCy}'])
for cat, gruppo in A_QC.groupby('Classe'):
    plt.scatter(gruppo[PCx-1], gruppo[PCy-1], label=cat)
for i in range(QC.shape[0]):
    plt.annotate(QC.loc[i,'Campione'], xy=(A_QC.iloc[i,(PCx-1)],A_QC.iloc[i,(PCy-1)]), xytext=(A_QC.iloc[i,(PCx-1)]*1.5,A_QC.iloc[i,(PCy-1)]*1.5), arrowprops=dict(arrowstyle='-', color='black', lw=2))
plt.title('Scores plot')
plt.xlabel(df_scores.columns[PCx - 1])
plt.ylabel(df_scores.columns[PCy - 1])
plt.legend(title='Classe')
plt.grid(True)
plt.show()